# parse logs to create state action reward from data
see (this document for further details)[https://docs.google.com/document/d/1dwnqLLsM3YizOFxxjOR1O6U3hEZhGllhJrQLBBqrK0c/edit]

## parse log in dataframe

In [11]:
import pandas as pd
import pickle
import env.constants as envconst

# read log data
with open('../raw data/training-log-corpus.log', 'r') as f:
    logs = f.readlines()

last_time = 0.0
last_student = ''
action_count = 0
l = []
for i, line in enumerate(logs):
    tokens = line.split('\n')[0]
    tokens = tokens.split('|')

    line_no = tokens[0]
    student_id = tokens[1]
    if len(student_id) != 8:
        print(
            f"ERROR: in line {i}, student id length should be 8, found {len(student_id)}!"
        )
        exit(0)

    action = tokens[2]

    # starting entries do not have time
    try:
        float(tokens[3])
    except:
        tokens.insert(3, 0.0)
    time = float(tokens[3])

    if last_student != student_id:
        last_time = 0.0
        # print(f"{action_count} actions found for student {last_student}")
        action_count = 0

    # check is time is almost monotonic
    if last_time - time >= 1:
        print(
            f"ERROR: in line {i}, time is not monotonic! current action {action}. total action found {action_count}. student {student_id}"
        )

    last_time = time
    last_student = student_id
    action_count += 1

    try:
        detail = tokens[4]
    except:
        detail = ""

    try:
        detail2 = tokens[5]
    except:
        detail2 = ""

    others = "|".join(tokens[6:])

    l.append({
        'line': line_no,
        'episode': student_id,
        'action': action,
        'time': time,
        'detail': detail,
        'more_detail': detail2,
        'others': others,
        'all': line.split('\n')[0]
    })

df = pd.DataFrame(l,
                  columns=['line', 'episode', 'action', 'time', 'detail', 'more_detail', 'others', 'all'])


# for students with "RESET"
# we are keeping if student reseted after planing signifcant part of the game (>70%)
# or resetted very early (<30%)
print("-- removing reset issues --")
dlt_idx = []
student_with_reset = df.loc[df['action'] == 'RESET']['episode'].unique()
for student in student_with_reset:
    df_s = df.loc[df['episode'] == student]
    total_steps = len(df_s)
    i = df.loc[(df['episode'] == student)
               & (df['action'] == 'RESET')].index[0]
    before_reset_steps = len(df_s.loc[:i])
    percent_before_reset = round(before_reset_steps / total_steps * 100, 2)
    drop_curr = []
    if percent_before_reset < 30.0:
        drop_curr = list(df_s.loc[df_s.index[0]:i + 1].index)
    elif percent_before_reset > 70.0:
        drop_curr = list(df_s.loc[i:].index)
    else:
        drop_curr = list(df_s.index)

    print(
        "student: {0} | total_steps: {1} | reset_at: {2}({3}%) | dropping: {4}"
        .format(student, total_steps, before_reset_steps, percent_before_reset,
                len(drop_curr)))
    dlt_idx += drop_curr


df = df.drop(index=dlt_idx).reset_index(drop=True)
df_raw = df.copy()
df_raw.to_pickle("../processed_data/raw_logs.pkl")

ERROR: in line 42330, time is not monotonic! current action RESET. total action found 190. student 100-0058
ERROR: in line 62840, time is not monotonic! current action RESET. total action found 1335. student 100-0082
ERROR: in line 64365, time is not monotonic! current action RESET. total action found 755. student 100-0084
ERROR: in line 89782, time is not monotonic! current action RESET. total action found 354. student 100-0122
ERROR: in line 104788, time is not monotonic! current action RESET. total action found 501. student 100-0142
ERROR: in line 117531, time is not monotonic! current action RESET. total action found 714. student 100-0159
ERROR: in line 142386, time is not monotonic! current action RESET. total action found 348. student 100-0195
ERROR: in line 211235, time is not monotonic! current action RESET. total action found 705. student 100-0307
ERROR: in line 223610, time is not monotonic! current action RESET. total action found 1021. student 100-0322
ERROR: in line 328796

In [12]:
# get student features
df_score = pd.read_csv('../raw data/training-survey-corpus.csv')
df_score = df_score[[
    'Student ID', 'Gender', 'Game-Playing Skill', 'Content Pre Total',
    'Normalized Learning Gain'
]]
df_score = df_score.rename(
    columns={
        'Student ID': 'episode',
        'Gender': 's_static_gender',
        'Game-Playing Skill': 's_static_gameskill',
        'Content Pre Total': 's_static_pretest',
        'Normalized Learning Gain': 'nlg'
    })

# splitting reward based on median
# few nlg are None!
df_score = df_score.loc[df_score['nlg']!='None']
df_score['nlg'] = pd.to_numeric(df_score['nlg'])
mid_nlg = df_score['nlg'].describe()['50%']
df_score.loc[df_score['nlg']<mid_nlg, 'nlg'] = -100.0
df_score.loc[df_score['nlg']>=mid_nlg, 'nlg'] = 100.0

mid_pretest = df_score['s_static_pretest'].describe()['50%']
df_score.loc[df_score['s_static_pretest']<mid_pretest, 's_static_pretest'] = 0.
df_score.loc[df_score['s_static_pretest']>=mid_pretest, 's_static_pretest'] = 1.

mid_skill = df_score['s_static_gameskill'].describe()['50%']
df_score.loc[df_score['s_static_gameskill']<mid_skill, 's_static_gameskill'] = 0.
df_score.loc[df_score['s_static_gameskill']>=mid_skill, 's_static_gameskill'] = 1.

df_score['s_static_gender'] = df_score['s_static_gender'] - 1.0

# making sure we have static features for all students
# 26 students are removed | total left 399
df = df.loc[df['episode'].isin(df_score['episode'].unique())]
df = df.reset_index(drop=True)

df = df.merge(df_score, on='episode', how='left')


## parse student trajectories

### load state names and their locations for student trajectories from the constants file | parse and save

In [13]:
from copy import deepcopy
import numpy as np

init_state = np.zeros(len(envconst.state_map))
sink_state = np.ones(len(envconst.state_map))

all_logs = []
step = 0
episode = ""
nlg = 0.0
def add_step(state, action_name, extra=None):
    global step, episode, all_logs, nlg
    # print(episode, step, action_name, extra)
    curr_state = deepcopy(state)
    curr_action = envconst.action_map[action_name]
    next_state = deepcopy(state)
    reward = 0.0
    done = False

    if 'a_talk_' in action_name:
        val = action_name.split('a_talk_')[-1]
        next_state[envconst.state_map['s_talk_'+val]] = 1

    elif 'a_workshsubmit' == action_name:
        next_state[envconst.state_map["s_workshsubmit"]] += 1
        next_state[envconst.state_map["s_solved"]] = extra # 1 when solved

    elif 'a_post' == action_name:
        next_state[envconst.state_map[extra]] = 1

    elif 'a_obj' == action_name:
        next_state[envconst.state_map[extra]] = 1

    elif 'a_label' == action_name:
        # labeling must not be complete
        if next_state[envconst.state_map["s_label_"+extra[0]]] == 0:
            next_state[envconst.state_map['s_label']] += 1
            # if solved, no longer available
            if extra[1] is True:
                next_state[envconst.state_map["s_label_"+extra[0]]] = 1

    elif 'a_objtest' == action_name:
        next_state[envconst.state_map['s_testleft']] -= 1
        if extra[0]!='s_objtest_null':
            next_state[envconst.state_map[extra[0]]] = 1

        if extra[1] is True:
            next_state[envconst.state_map['s_testpos']] = 1
            # next_state[envconst.state_map["s_label_slide"]] = 1

        if next_state[envconst.state_map['s_testleft']] < 0:
            print("ERROR! Test left is negative!")

    elif 'a_book' == action_name:
        next_state[envconst.state_map['s_book_'+extra]] = 1

    elif 'a_end' == action_name:
        next_state[envconst.state_map['s_end']] = 1
        reward = nlg
        done = True

    else:
        state_name = "s" + action_name[1:]
        next_state[envconst.state_map[state_name]] += 1

    all_logs.append({'episode':episode, 'step': step, 'state': curr_state,
                     'action': curr_action, 'reward': reward,
                     'next_state': next_state, 'done': done, 'nlg': nlg, 'info': {'extra': extra, 'end': False}})
    step += 1
    return deepcopy(next_state)

skip_till_next = False
state = deepcopy(init_state)
for idx, row in df.iterrows():
    if episode != row['episode']:
        if episode != "" and skip_till_next == False:
            # finished early without any gameover
            state = add_step(state, 'a_end')  # game ends

        episode = row['episode']
        step = 0
        skip_till_next = False
        state = deepcopy(init_state)
        state[envconst.state_map['s_static_gender']] = row['s_static_gender']
        state[envconst.state_map['s_static_pretest']] = row['s_static_pretest']
        state[envconst.state_map['s_static_gameskill']] = row['s_static_gameskill']
        nlg = row['nlg']

    if skip_till_next is True:
        continue

    if step == envconst.max_ep_len-1:
        # cut off at horizon
        state = add_step(state, 'a_end')
        skip_till_next = True
        continue


    if row['action']=='SOLUTION':
        disease, item = row['detail'].split("-")[1], row['more_detail'].split("-")[1]
        if disease == "Influenza":
            state[envconst.state_map['s_target_disease']] = 0
        elif disease == "Salmonellosis":
            state[envconst.state_map['s_target_disease']] = 1
        else:
            print("ERROR! Unknown target disease", disease, row['all'])
            break
        if item == "Egg":
            state[envconst.state_map['s_target_item']] = 0
        elif item == "Milk":
            state[envconst.state_map['s_target_item']] = 1
        elif item == "Sandwich":
            state[envconst.state_map['s_target_item']] = 2
        else:
            print("ERROR! Unknown target item", item, row['all'])
            break
        continue

    if row['action'] == 'ADAPTATION':
        if row['detail'] in ['select-kim-reveal', 'select-direct-toward-goal', 'select-record-finding', 'select-increase-urgency', 'select-reflection-prompt', 'select-offtask-discourage']:
            # kim redirects to see quentine. not important
            continue

        if row['detail'] == 'select-test-count':
            # this is not an aes i am tracking
            if row['others'] == 'selected-1':
                state[envconst.state_map['s_testleft']] = 3
            elif row['others'] == 'selected-2':
                state[envconst.state_map['s_testleft']] = 5
            elif row['others'] == 'selected-3':
                state[envconst.state_map['s_testleft']] = 10
            continue
        if row['detail'] == 'select-present-quiz':
            val = row['others'].split('-')[-1]
            aes = "s_aes_kno_" + val
            state[envconst.state_map[aes]] += 1
            continue
        if row['detail'] == 'select-bryce-reveal':
            val = row['others'].split('-')[-1]
            aes = "s_aes_pas_" + val
            state[envconst.state_map[aes]] += 1
            continue
        aes_name = row['detail'].split('-')[1][:3]
        val = row['others'].split('-')[-1]
        aes = "s_aes_" + aes_name + "_" + val
        state[envconst.state_map[aes]] += 1
        continue

    # ignore logs not relevant
    if row['action'] in ['AUTHENTICATE', 'PDAOPEN', 'PDACLOSE', 'OPEN', 'STUDY', 'ADDPOINTS', 'GOALCOMPLETE', 'NOTIFICATION', 'SELF-REPORT', 'LOOKEND', 'DROP', 'STOWITEM', 'RETRIEVEITEM', 'FINALSCORE', 'READ', 'CLOSE', 'STUDENTENTRY', 'COORGET', 'OPEN']:
        continue

    if row['action'] == "TALK":
        who = row['more_detail'].split('cur-action-talk-')[1]
        if "extra" in who:
            who = who[:-1]
        state = add_step(state, 'a_talk_'+who)
        continue

    if row['action'] == "DIALOG":
        if row['more_detail']=="kim":
            if row['others'] == "Hmm,Iamnotsurethisisquiteright.":
                state = add_step(state, 'a_workshsubmit', extra=0)  # continue
            elif row['others'] in ["Greatjob,yousolvedourmystery!", "Thisdiagnosismakesperfectsense!We'llstartthetreatmentplanrightaway."]:
                # only 5 cases like this
                if state[envconst.state_map['s_testpos']]==0:
                    target_item = state[envconst.state_map['s_target_item']]
                    if target_item == 0:
                        item = "s_obj_egg"
                    elif target_item == 1:
                        item = "s_obj_milk"
                    else:
                        item = "s_obj_sandwich"
                    found = True
                    # state = add_step(state, 'a_objtest', extra=(item, found))
                    #
                    # # add slide as well
                    # state = add_step(state, 'a_label', extra=("slide", True))

                state = add_step(state, 'a_workshsubmit', extra=1)  # solved, ended, no more change
                skip_till_next = True
        continue

    if row['action'] == "LOOKSTART":
        post = row['detail'].split('-')
        if len(post)>=3:
            post = "s_post_" + post[1] + "_" + post[2]
        elif len(post)==2:
            post = "s_post_" + post[1]
        else:
            print("ERROR! wrong poster name", row['all'])
            break
        state = add_step(state, 'a_post', extra=post)
        continue

    if row['action'] == "PICKUP":
        item = row['more_detail'].split('-')[-1]
        if item in ["1", "3", "null"]:
            continue
        elif item in ["10", "11", "4"]:
            item = "s_obj_jar" + item
        else:
            item = "s_obj_" + item

        state = add_step(state, 'a_obj', extra=item)
        continue

    if row['action'] == "NOTES":
        state = add_step(state, 'a_notetake')
        continue

    if row['action'] == "PDAUSE":
        if row['detail'] in ["notepad", "micromanual"]:
            state = add_step(state, 'a_noteview')
            continue
        elif row['detail'] in ['mainscreen', 'add-note', 'notify', 'take-quiz-prompt', 'pdamsg', 'pdamsg-quiz2', 'message-panel']:
            continue
        else:
            print("WHAT!", idx, row['all'])

    if row['action'] == "LABELING":
        type = row['detail']
        correct = row['others'].split("|")[-1]
        correct = True if correct=="correct" else False
        state = add_step(state, 'a_label', extra=(type, correct))
        continue

    if row['action'] == "TESTCOMPUTER":
        test_remaining = int(row['others'].split('|')[2].split('-')[1])
        if state[envconst.state_map['s_testleft']] != test_remaining:
            # there is a bug in adding new tests. sometimes added-test log is missed
            print("ERROR! Wrong number of test remains", row['all'])
            state[envconst.state_map['s_testleft']] = test_remaining
        continue

    if row['action'] == "TESTOBJECT":
        if row['detail'] in ["MultipleObjects", "NoObject", "object-null"] or row['more_detail'] == "noenergy":
            continue

        item = row['detail'].split('-')[-1]
        if item in ["10", "11", "4"]:
            item = "s_objtest_jar" + item
        else:
            item = "s_objtest_" + item

        found = False
        if row['more_detail'] == "correct":
            found = True

        state = add_step(state, 'a_objtest', extra=(item, found))
        continue

    if row['action'] == "QUIZ":
        if row['detail'] == 'quiz-answer':
            if 'added-test' in row['others']:
                state = add_step(state, 'a_testleft')
            continue
        else:
            continue

    if row['action'] == "BOOKREAD":
        book = row['detail'][:-4]
        state = add_step(state, action_name="a_book", extra=book)
        continue

    if row['action'] == "BRYCECOMPUTER":
        state = add_step(state, action_name="a_computer")
        continue

    if row['action'] == "WORKSHEET":
        if row['detail'] == "close-worksheet":
            state = add_step(state, action_name="a_worksheet")
        continue

    if row['action'] == "GAMEOVER":
        state = add_step(state, 'a_end')  # game ends
        skip_till_next = True
        continue

df_logs = pd.DataFrame(all_logs)
df_logs.loc[df_logs['done']==False, 'nlg'] = 0.0

dd = df_logs.groupby(['episode']).count()
df_logs = df_logs.loc[df_logs['episode'].isin(dd.loc[dd['step']>=envconst.min_ep_len].index)]
df_logs = df_logs.sort_values(by=['episode', 'step']).reset_index(drop=True)

df_logs.to_pickle('../processed_data/student_trajectories.pkl')

ERROR! Wrong number of test remains 7899|100-0011|TESTCOMPUTER|2452.080|run-test|2447.699951|2452.085938|4.385986|testsremaining-3|Hypothesis:ContaminatedwithCarcinogens|Reason:Sickmembersdrankit
ERROR! Wrong number of test remains 8132|100-0011|TESTCOMPUTER|2969.250|run-test|2963.784424|2969.248535|5.464111|testsremaining-1|Hypothesis:ContaminatedwithMutagens|Reason:Sickmembersdrankit
ERROR! Wrong number of test remains 8144|100-0011|TESTCOMPUTER|2991.810|run-test|2987.909912|2991.795410|3.885498|testsremaining-1|Hypothesis:ContaminatedwithCarcinogens|Reason:Sickmembersdrankit
ERROR! Wrong number of test remains 8179|100-0011|TESTCOMPUTER|3091.530|run-test|3084.674072|3091.530029|6.855957|testsremaining-1|Hypothesis:ContaminatedwithPathogens|Reason:Sickmembersateit
ERROR! Wrong number of test remains 8208|100-0011|TESTCOMPUTER|3152.010|run-test|3144.575928|3152.002197|7.426270|testsremaining-1|Hypothesis:ContaminatedwithCarcinogens|Reason:Sickmembersdrankit
ERROR! Wrong number of test

In [14]:
df_logs

,episode,step,state,action,reward,next_state,done,nlg,info
0,100-0001,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4,0.0,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",False,0.0,"{'extra': None, 'end': False}"
1,100-0001,1,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",11,0.0,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",False,0.0,"{'extra': 's_post_scientific_method', 'end': F..."
2,100-0001,2,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",11,0.0,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",False,0.0,"{'extra': 's_post_scientific_method', 'end': F..."
3,100-0001,3,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",11,0.0,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",False,0.0,"{'extra': 's_post_scientific_method', 'end': F..."
4,100-0001,4,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",11,0.0,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",False,0.0,"{'extra': 's_post_prevention_treatment', 'end'..."
...,...,...,...,...,...,...,...,...,...
65569,100-0828,197,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",15,0.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",False,0.0,"{'extra': None, 'end': False}"
65570,100-0828,198,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",11,0.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",False,0.0,"{'extra': 's_post_prevention_treatment', 'end'..."
65571,100-0828,199,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",4,0.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",False,0.0,"{'extra': None, 'end': False}"
65572,100-0828,200,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",18,0.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",False,0.0,"{'extra': 0, 'end': False}"


## parse narrative trajectories

### define state action for narrative trajectories

In [15]:
def get_aes_trajectories(df, target_aes, state_map=envconst.state_map):
    aes_state_map = {fname: fid for fname, fid in state_map.items() if target_aes in fname}
    step = 0
    episode = ""
    logs = []

    state = None
    action = None
    prev_row = deepcopy(df.iloc[0])

    for i, row in df.iterrows():
        if row['episode'] != episode:
            if state is not None:
                # lazy add
                logs.append({'episode':episode, 'step': step, 'state': state, 'action': action, 'reward': prev_row['reward'], 'next_state': deepcopy(prev_row['state']), 'done': prev_row['done'], 'nlg': prev_row['nlg']})
                step = 0
                state = None

                prev_row = deepcopy(row)
            episode = row['episode']

        for fname, fid in aes_state_map.items():
            if row['state'][fid] != prev_row['state'][fid]:
                if state is not None:
                    # lazily waiting for next state
                    logs.append({'episode':episode, 'step': step, 'state': state, 'action': action, 'reward': prev_row['reward'], 'next_state': deepcopy(prev_row['state']), 'done': prev_row['done'], 'nlg': prev_row['nlg']})
                    step += 1

                state = deepcopy(prev_row['state'])
                action = int(fname.split(target_aes)[1]) - 1

        prev_row = deepcopy(row)

    if state is not None:
        # lazy add
        logs.append({'episode':episode, 'step': step, 'state': state, 'action': action, 'reward': prev_row['reward'], 'next_state': deepcopy(prev_row['state']), 'done': prev_row['done'], 'nlg': prev_row['nlg']})

    return pd.DataFrame(logs)

# gen narrative data
target_aeses = ["s_aes_bry_", "s_aes_ter_", "s_aes_que_", "s_aes_kno_", "s_aes_wor_", "s_aes_pas_"]
all_aes_trajectories = {}
for target_aes in target_aeses:
    df_aes = get_aes_trajectories(df_logs, target_aes)
    all_aes_trajectories[target_aes[2:-1]] = deepcopy(df_aes)

pickle.dump(all_aes_trajectories, open('../processed_data/narrative_trajectories.pkl', 'wb'))

# random

In [1]:
import pandas as pd
import utils
import env.constants as envconst
import numpy as np
df_org = pd.read_pickle('../processed_data/student_trajectories.pkl')
df_org.apply(lambda x: utils.state_string(x['state']), axis=1)

2022-12-22 13:47:02.889287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


0        I am a female. I have low pretest and low game...
1        I am a female. I have low pretest and low game...
2        I am a female. I have low pretest and low game...
3        I am a female. I have low pretest and low game...
4        I am a female. I have low pretest and low game...
                               ...                        
65569    I am a male. I have low pretest and high game ...
65570    I am a male. I have low pretest and high game ...
65571    I am a male. I have low pretest and high game ...
65572    I am a male. I have low pretest and high game ...
65573    I am a male. I have low pretest and high game ...
Length: 65574, dtype: object

In [17]:
df_raw = pd.read_pickle("../processed_data/raw_logs.pkl")
d = df_raw.loc[(df_raw['action']=='TESTOBJECT')]
d.groupby(['detail']).count()

,line,episode,action,time,more_detail,others,all
detail,,,,,,,
MultipleObjects,119,119,119,119,119,119,119
NoObject,352,352,352,352,352,352,352
object-cactus,20,20,20,20,20,20,20
object-food-apple,66,66,66,66,66,66,66
object-food-bananas,551,551,551,551,551,551,551
object-food-bread,767,767,767,767,767,767,767
object-food-cheese,113,113,113,113,113,113,113
object-food-coconut,357,357,357,357,357,357,357
object-food-egg,654,654,654,654,654,654,654
